# Tagfuse File System Feature Test

This file contains a set of tests that verify the features of the Tag Fuse Filesystem.

## Prerequisites

- Raspberry Pi with latest software (See ```01_RPi_Build.md``` for details on contructing and installing a new RPi Disk Image). Make sure that the lates TagNet Basestation Software is installed, too.
- Tag with latest software (see ```?``` for details on constructing and installing a new Tag Image)

## Initial State

The following variables provide user environment control.

In [13]:
TREE_BASE    = '/home/pi/tags'
nid='658bc8e5205c'
nid='a0b696b4fb29'
TAG_ID       = TREE_BASE + '/' + nid
TAG_BASE     = TAG_ID + '/tag'
IMAGE_BASE   = TAG_BASE + '/sd/0/img'
DBLK_BASE    = TAG_BASE + '/sd/0/dblk'
SYS_BASE     = TAG_BASE + '/sys'
POLL_BASE    = TAG_BASE + '/poll'
SPARSE_STORE = '/home/pi/Desktop'
IMAGE_STORE  = '/mnt/neptune/tagbin'

## NOTES / ISSUES
- writing image when no room available gets stuck in a loop - very bad

## Install Tagfuse Driver

First, get the tagfuse driver running as a daemon

In [7]:
!tagfuse -b -s $SPARSE_STORE $TREE_BASE

TagNet Fuse Driver version: 0.2.2
('mountpoint', '/home/pi/tags')
('verbosity turned on', 1)
('sparse files stored here', '/home/pi/Desktop')
Si446x Driver Version: 0.3.7
TagNet Driver Version 0.1.3
loading sparsefile


Now look at the entire tree of information available in the tag

In [36]:
!tree -a $TREE_BASE
#!tree -aJsD $TREE_BASE

/home/pi/tags
├── a0b696b4fb29
│   └── tag
│       ├── info
│       │   └── sens
│       │       └── gps
│       │           ├── cmd
│       │           └── xyz
│       ├── poll
│       │   ├── cnt
│       │   └── ev
│       ├── sd
│       │   └── 0
│       │       ├── dblk
│       │       │   ├── byte
│       │       │   ├── .committed
│       │       │   ├── .last_rec
│       │       │   ├── .last_sync
│       │       │   ├── note
│       │       │   └── .recnum
│       │       ├── img
│       │       └── panic
│       │           ├── byte
│       │           └── .count
│       ├── sys
│       │   ├── active
│       │   ├── backup
│       │   ├── golden
│       │   │   └── 0.3.600
│       │   ├── nib
│       │   │   └── 255.255.65535
│       │   └── running
│       │       └── 0.3.600
│       └── .test
│           ├── drop
│           ├── echo
│           ├── ones
│           ├── sum
│           └── zeros
├── ffffffffffff
│   └── tag
│       ├── info
│       │   └── sens
│       │   

In [12]:
!ls -al $TREE_BASE

total 4
drwxr-x--x  6 pi pi    0 Dec 31  1969 .
drwxr-xr-x 30 pi pi 4096 Jul 16 10:07 ..
drwxr-x--x  3 pi pi    0 Dec 31  1969 a0b696b4fb29
drwxr-x--x  3 pi pi    0 Dec 31  1969 ffffffffffff
drwxr-x--x  4 pi pi    0 Dec 31  1969 .poll
drwxr-x--x  6 pi pi    0 Dec 31  1969 .test


## Examine the Dblk Area

In [ ]:
#STOP

In [14]:
!ls -al $DBLK_BASE

total 0
drwxr-x--x 8 pi pi        0 Dec 31  1969 .
drwxr-x--x 5 pi pi        0 Dec 31  1969 ..
-r--r--r-- 1 pi pi 37618948 Jul 16 15:38 byte
-r--r--r-- 1 pi pi 37619200 Jul 16 15:39 .committed
-r--r--r-- 1 pi pi 37619028 Jul 16 15:39 .last_rec
-r--r--r-- 1 pi pi 37615624 Jul 16 15:39 .last_sync
-rw-rw---- 1 pi pi        0 Jul 16 15:38 note
-r--r--r-- 1 pi pi    17287 Jul 16 15:38 .recnum


In [15]:
!echo `stat -c '%s' $DBLK_BASE/.last_rec`

37619856


In [16]:
!tagdump -n 10 --net -j `stat -c '%s' $DBLK_BASE/.last_sync` $DBLK_BASE/byte

---  offset    recnum    rtime    len  dt  name
---                      0.000000 2018/7/16 22:00 (0) UTC
--- @37619788   17306 2469.170562  28   3  SYNC  prev: @37615624 (0x23df808)
--- @37619816   17307 2469.193206  40   4  EVENT unk 7 4 0 4
--- @37619856   17308 2469.193420  40   4  EVENT UNDOCKED 7 4292264376 4290646198 3821812
--- @37619896   17309 2470.170257  40   4  EVENT GPS_GEO 132253456 5671 9000 0
--- @37619936   17310 2470.170501  40   4  EVENT DOCKED 370489592 3074700518 2010 167964000
--- @37619976   17311 2470.193603  40   4  EVENT UNDOCKED 7 4292264376 4290646198 3821812
--- @37620016   17312 2471.170471  40   4  EVENT GPS_GEO 132253456 5671 10000 0
--- @37620056   17313 2471.170715  40   4  EVENT DOCKED 370489592 3074700519 2010 167965000
--- @37620096   17314 2471.193817  40   4  EVENT UNDOCKED 7 4292264376 4290646198 3821812
--- @37620136   17315 2472.170745  40   4  EVENT GPS_GEO 132253456 5671 11000 0

*** end of processing @37620176 (0x23e09d0),  processed: 10 re

In [ ]:
# !tagdump -j `stat -c '%s' $DBLK_BASE/.last_rec` $DBLK_BASE/byte

## Write a note into the Dblk Area

In [17]:
!echo 'this is a note' > $DBLK_BASE/note

In [18]:
ls -l $DBLK_BASE/note

-rw-rw---- 1 pi pi 1 Jul 16 15:39 /home/pi/tags/a0b696b4fb29/tag/sd/0/dblk/note


In [19]:
!echo 'this is a second note' > $DBLK_BASE/note

In [20]:
ls -l $DBLK_BASE/note

-rw-rw---- 1 pi pi 2 Jul 16 15:39 /home/pi/tags/a0b696b4fb29/tag/sd/0/dblk/note


## Extract a Panic Event

## Get GPS Position

## Send Commands to control the GPS

## Examine Pending Events on the Tag

In [21]:
!ls -l $POLL_BASE

total 0
-r--r--r-- 1 pi pi 3 Jul 16 15:40 cnt
-r--r--r-- 1 pi pi 3 Jul 16 15:40 ev


## Show the state of software currently installed on the tag

In [22]:
!tree $SYS_BASE

/home/pi/tags/a0b696b4fb29/tag/sys
├── active
├── backup
├── golden
│   └── 0.3.598
├── nib
│   └── 255.255.65535
└── running
    └── 0.3.598

5 directories, 3 files


## Change the state of software on the tag

Check to see which software image is currently active (loaded into the Flash NIB space)

In [24]:
!ls $SYS_BASE/active

Set the active version NOTE: REBOOTS TAG

In [25]:
!ln $IMAGE_BASE/0.3.461 $SYS_BASE/active/0.3.461

ln: failed to access '/home/pi/tags/a0b696b4fb29/tag/sd/0/img/0.3.461': No such file or directory


In [26]:
!ls -l $SYS_BASE/active

total 0


Try again with same version faile

In [ ]:
!ln $IMAGE_BASE/0.3.460 $SYS_BASE/active/0.3.460

Set the backup version

In [ ]:
!ln $IMAGE_BASE/0.3.461 $SYS_BASE/backup/0.3.461

In [ ]:
!ls $SYS_BASE/backup

In [ ]:
!tree $SYS_BASE

Reboot into the NIB

Reboot into Golden

## Load new Image onto Tag

In [ ]:
#STOP

Possible software images ready for loading are in the IMAGE_STORE directory

In [37]:
!ls -l $IMAGE_STORE/*

-rwxrwxrwx 1 pi devgrp  63188 May 29 23:51 /mnt/neptune/tagbin/0.3.461
-rwxrwxrwx 1 pi devgrp  94164 Jun 28 10:30 /mnt/neptune/tagbin/0.3.574
-rwxrwxrwx 1 pi devgrp  94900 Jul 16 15:54 /mnt/neptune/tagbin/0.3.599
-rwxrwxrwx 1 pi devgrp  24212 Feb  8 21:58 /mnt/neptune/tagbin/blink464.bin
-rwxrwxrwx 1 pi devgrp  20944 Jan 23 15:49 /mnt/neptune/tagbin/main2.bin
-rwxrwxrwx 1 pi devgrp  45976 Jan 18 01:25 /mnt/neptune/tagbin/main386.bin
-rwxrwxrwx 1 pi devgrp  47088 Feb  3 23:08 /mnt/neptune/tagbin/main442.bin
-rwxrwxrwx 1 pi devgrp  47088 Feb  3 23:08 /mnt/neptune/tagbin/main443.bin
-rwxrwxrwx 1 pi devgrp  48580 Feb 26 00:47 /mnt/neptune/tagbin/main496.bin
-rwxrwxrwx 1 pi devgrp 927726 Jun 23 19:11 /mnt/neptune/tagbin/main566.bin
-rwxrwxrwx 1 pi devgrp  20944 Feb 26 00:38 /mnt/neptune/tagbin/main.bin


See what is currently in the tag's Image storage

In [28]:
!ls -l $IMAGE_BASE

total 0


Now remove all images currently on the tag. NOTE: the active and backup images cannot be deleted (see later steps for how to find out which version is active)

In [29]:
!rm $IMAGE_BASE/*

rm: cannot remove '/home/pi/tags/a0b696b4fb29/tag/sd/0/img/*': No such file or directory


Check the tag's Image storage again to verify

In [30]:
!ls -l $IMAGE_BASE

total 0


Use `cp` to copy one of the software images in the IMAGE_STORE to the tag
Monitor progress using pv

In [51]:
version = '0.3.599'
#!cp main386.bin $IMAGE_BASE/0.2.386 | pv
!pv  -n $IMAGE_STORE/$version > $IMAGE_BASE/$version

0
pv: write failed: No such file or directory


Check to see that image is now on the tag

In [ ]:
!ls -l $IMAGE_BASE

Load another image to the tag, this time using `dd`

In [ ]:
!dd if=$IMAGE_STORE/main442.bin of=$IMAGE_BASE/0.2.442

In [ ]:
!tree $IMAGE_BASE

In [ ]:
!dd if=$IMAGE_STORE/main443.bin of=$IMAGE_BASE/0.2.443

In [ ]:
!ls -l $IMAGE_BASE

In [ ]:
!dd if==%IMAGE_STORE/blink464.bin of=$IMAGE_BASE/0.2.464

In [ ]:
!ls -l $IMAGE_BASE

Check for memory full error condition

In [ ]:
!dd if=$IMAGE_STORE/main496.bin of=$IMAGE_BASE/0.2.496

In [ ]:
!ls -l $IMAGE_BASE

## Stop the Fuse Driver

In [ ]:
!fusermount -u ~/tags